# Web Scraping adn Data Pipeline

* ขั้นตอนและกระบวนการที่เขียนโค้ดขึ้นมานั้น มีความแตกต่างกันไปในแต่ละ website โครงสร้างการ scraping จะเปลี่ยนไปตามโครงสร้างนั้น โปรดศึกษาความเข้าในการ scraping ด้วยตนเองก่อนนำไปทดลอง
* การดึงมาใช้งาน บางเว็บไซต์ <ins>**จำเป็น**</ins>ต้องขออนุญาติในการเก็บข้อมูล ควรทำการเก็บข้อมูลอย่างระมัดระวัง
* ผู้จัดทำไม่อนุญาติให้นำไปใช้ในด้านพาณิชย์ทุกรูปแบบ จัดทำขึ้นเพื่อใช้ในการทดสอบความสามารถขอผู้จัดทำในการสมัครงานเท่านั้น
* การใช้งาน Hugging Face Model บน cloud server หรือบนเครื่องที่ deploy เอาไว้ ขอ<ins>**สงวน**</ins>ช่องทางการใช้งานไว้


## Dependencies

In [1]:
import requests
import json
import time
from IPython.display import Markdown, display
from pathlib import Path

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from openai import OpenAI

In [ ]:
import os 
print("Where am I:", os.getcwd())

## Functions

In [2]:
def show_prettify(just_object: BeautifulSoup):
    # แปลง HTML ที่ได้จาก BeautifulSoup ให้อยู่ในรูป pretty (indent สวยงาม) แล้ว print ออกมา
    print(just_object.prettify())


def get_content_in_section(section_soup: BeautifulSoup) -> str:
    # ดึงเนื้อหาภายใน div ตัวที่ 9 (index = 8) ของ section ที่สนใจ
    just_object = section_soup.find_all('div')[8]

    # แปลง tag <strong> ให้เป็น Markdown รูปแบบ **bold**
    for tag in just_object.find_all('strong'):
        tag.replace_with(f'**{tag.get_text()}**')

    # แปลง list item (<li>) ให้ขึ้นบรรทัดใหม่ด้วย \n
    for li in just_object.find_all('li'):
        li.replace_with(f'\n{li.get_text()}')

    # ใช้ตรวจสอบโครงสร้าง heading (เช่น h1, h2, h3) แล้วทำเป็น Markdown header
    old_num_head = float('inf')      # เก็บลำดับ heading ล่าสุดที่เคยเจอ
    num_sharp = 0                   # จำนวนเครื่องหมาย # ที่จะใช้ทำ Markdown header
    content = ''                    # เก็บข้อความที่สร้างขึ้น

    # วน loop ผ่าน element ภายใน div
    for idx, obj in enumerate(just_object):
        obj_name: str = obj.name
        text: str = "".join(obj.strings)    # รวมข้อความภายใน tag
        # ถ้า element เป็น heading (เช่น h1, h2, h3 ...)
        if obj_name.startswith('h'):
            # ถ้าเป็น heading ใหม่ที่ลึกกว่า ให้เพิ่มจำนวนเครื่องหมาย #
            if num_sharp == 0 or int(obj_name[1]) > old_num_head:
                old_num_head = int(obj_name[1])
                num_sharp += 1
            text = "#" * num_sharp + " " + text
        content += text + '\n'

    # คืนค่าข้อความทั้งหมด (ตัดช่องว่าง/บรรทัดเกินออกด้วย strip)
    return content.strip()

def get_information(soup: BeautifulSoup):
    # เข้าถึง tag <main> และ <section> ของหน้า HTML
    main_soup = soup.find('main')
    section_soup = main_soup.find('section')

    # ดึงหัวข้อหลักจาก <h1>
    topic = section_soup.find('h1').string

    # ดึง tags จาก div ลำดับที่ 3 (index = 2)
    tags = [tag.string for tag in section_soup.find_all('div')[2]]

    # สร้าง dict เก็บข้อมูล: หัวข้อ, แท็ก, เนื้อหา
    info = dict(
        topic=topic,
        tags=tags,
        content=get_content_in_section(section_soup),
    )
    return info

def get_url_from(link: str, pattern: str = "a[href*='\.\./en/ideas/']"):
    # เปิดหน้าเว็บด้วย Selenium, ChromeDriver
    # *** เปลี่ยน browser ตามที่คุณต้องการได้เลย แต่ผมแนะนำ Chrome ง่ายและสะดวก แต่อาจกิน resource เครื่องได้
    driver = webdriver.Chrome()
    driver.get(link)

    # รอให้ elements โหลดเสร็จ (สูงสุด 10 วินาที)
    driver.implicitly_wait(10)

    # แปลงหน้าเว็บเป็น soup object
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # เลือก element ตาม pattern แล้วดึง href ออกมา
    links = [a['href'] for a in soup.select(pattern)]

    # จำเป็นมาก - ปิด browser ไม่ browser จะเปิดค้างเอาไว้ เปลือง resource ของ local
    driver.quit()
    return links


## Scraping URLs with Selenium and BeautifulSoup

In [3]:
# กำหนด URL ตัวอย่างที่ต้องการดึงข้อมูล (หน้าเว็บจริงที่จะนำมา parse)
example_url = 'https://www.jenosize.com/en/ideas/understand-people-and-consumer/getting-to-know-org-chart-and-structures'

# ใช้ requests เพื่อดึงข้อมูล HTML ของหน้าเว็บจาก URL ข้างต้น
url_content = requests.get(example_url)

# แปลงข้อมูล HTML ที่ได้มาเป็น BeautifulSoup object
# ทำให้เราสามารถเข้าถึง element ภายใน HTML ได้ง่าย (เช่นค้นหาแท็ก <div>, <h1> เป็นต้น)
soup = BeautifulSoup(url_content.content)


In [4]:
# เข้าถึงโครงสร้าง HTML หลักของหน้าเว็บ โดยเลือก <main>
main_soup = soup.find('main')

# ดึง section ภายใน <main> ที่ต้องการใช้งาน
section_soup = main_soup.find('section')

# ใช้ฟังก์ชัน get_content_in_section (ที่เราสร้างไว้ก่อนหน้า)
# เพื่อแปลงข้อมูลภายใน section ให้เป็นข้อความในรูป Markdown
content = get_content_in_section(section_soup)

# แสดงผลข้อความที่ได้ในรูปแบบ Markdown บน Notebook
display(Markdown(content))


# Managing Organizations with Organization Chart & Structure

To effectively manage an organization, it is essential to rely on clear planning and organizational structures to ensure smooth operations and achieve organizational goals. In this article, we will introduce you to a key tool that helps improve organizational management: the "**Organization Chart & Structure.**"

## What is Organization Chart & Structure ?

**The Organization Chart & Structure, also known as Org Chart, is a visual representation of an organization's management structure and hierarchy of positions**. It helps to understand the distribution of authority, responsibilities, and roles of each department and individual within the organization clearly. Essentially, the Organization Chart acts as a compass that guides employees to better understand their roles, know who they report to, and collaborate effectively with others.

## Types Organization Chart & Structure

Typically, organization charts come in various types, each suitable for different organizational structures and work formats. Here, we'll discuss three main types commonly used by many organizations:

**Hierarchical Chart**: This is the traditional structure, displaying a top-down hierarchy. It's suitable for large organizations with clear departmental divisions and job roles.
**Flat Chart**: This structure is horizontal, reducing hierarchical levels and emphasizing collaboration. It's suitable for small organizations with high flexibility.
**Matrix Chart**: A blended structure where employees have both primary reporting lines and secondary lines reporting to project managers. It's suitable for organizations with cross-functional projects.

## The Components of Organization Chart & Structure


**Hierarchical Levels**: These represent the levels of authority within the organization, ranging from top executives down to operational staff.
**Departments/Divisions**: Departments or divisions depict the segmentation of tasks and responsibilities according to different functions, such as Marketing, Accounting and Finance, Human Resources, etc.
**Job Titles: Job titles** specify the positions and roles of each employee within the organization.
**Lines of Authority**: These illustrate the relationships and reporting lines between superiors and subordinates, outlining the flow of authority and responsibility.

However, designing a good organization chart & structure should consider important principles such as clarity, comprehensiveness, avoiding duplication of roles, and effective communication. Additionally, there should be regular reviews and adjustments to the organizational chart to align with changes and expansions in the organization's future.

## The Benefits of Using Organization Chart & Structure?

### Help to clarify the organizational structure

**Employees understand their roles, responsibilities, and accountabilities**: Because the org chart clearly shows what tasks each department is responsible for, who they report to, and who they need to collaborate with.
**Employees understand the hierarchy within the organization**: Knowing who is who, who heads which department, and who has decision-making authority.
**Help employees work with purpose**: When they understand their roles, responsibilities, and accountabilities, they can work with a purpose aligned with the organization's goals.

### Enhance organizational efficiency

**Facilitate effective communication**: A clear organization structure promotes efficient communication flow because it eliminates unnecessary steps and bureaucracy. This enables more effective communication among individuals within the organization.
**Promote coordination between departments or divisions**: By illustrating how departments or divisions relate to each other, the org chart helps teams coordinate smoothly. It shows how they are interconnected, facilitating efficient collaboration.
**Accelerate goal achievement**: When communication and coordination are efficient, teamwork thrives. This leads to quicker goal achievement as tasks are completed more effectively and efficiently.

### Facilitate organizational growth

**Support organizational scalability**: Having an organization chart & structure serves as a tool to assess the current state of the organizational structure and determine if the organization can expand further. If any areas of the structure require improvement to better adapt to market conditions and business needs, adjustments can be made promptly.
**Smoothen business expansion**: With an organization chart & structure defining roles and responsibilities, each employee knows their responsibilities, reducing redundancy in tasks. This enables business expansion to proceed smoothly and efficiently, with minimal obstacles.

### Assist in HRM (Human Resource Management)
HRM involves managing human resources, and designing an organization chart & structure aids HRM in the following ways:

**Facilitate workforce planning**: The organization chart & structure provides HR with an overview of the organizational structure, allowing them to analyze workforce needs, plan recruitment, development, and retention strategies effectively.
**Enhance performance evaluation**: A clear organization structure enables HR to set Key Performance Indicators (KPIs) and evaluate employee performance more accurately.
**Manage compensation**: Having an org chart allows HR to establish appropriate salary structures and benefits packages tailored to each position within the organization.

### Assist in HRD (Human Resource Development)
While HRM involves managing services, HRD focuses on the "development" of human resources, and organization chart & structure are equally beneficial for this aspect of work. For instance:

**Facilitate effective communication between HR and employees**: In a company with a flat organizational structure, HR can choose communication channels that best fit the organization structure, such as email or MS Teams.
**Employees feel a sense of future with the organization**: An Organization Structure conducive to development provides employees with opportunities for learning, skill development, and career growth, fostering a more conducive environment for personal and professional development compared to organizations with unclear structures.

Choosing the appropriate organization chart & structure depends on various factors such as the size of the organization, the nature of products or services, the business environment, and even the organization's strategy. Additionally, considerations should be given to interdepartmental coordination, power distribution in decision-making, and internal communication to ensure smooth operations and efficient goal attainment.

To improve the organization, it's crucial to start with internal management. And **Jenosize** can assist you! We offer "HappyWork," a human resource management system for HR and modern organizations. With features tailored to HRM and HRD tasks, HappyWork makes HR management easier than ever before. Try it for free today!

<h3>จุดสังเกตที่น่าสนใจ</h3>

- ถ้าหากดเราเปิดดู raw_text จะเห็นว่า markdown ถูกกำกับด้วย**สัญลักษณ์ (symbol)** มากมาย เราใช้ประโยชน์จากสิ่งนั้นในการแบ่งเนื้อหาได้


In [5]:
print(content[:1000] + ' ...')

# Managing Organizations with Organization Chart & Structure

To effectively manage an organization, it is essential to rely on clear planning and organizational structures to ensure smooth operations and achieve organizational goals. In this article, we will introduce you to a key tool that helps improve organizational management: the "**Organization Chart & Structure.**"

## What is Organization Chart & Structure ?

**The Organization Chart & Structure, also known as Org Chart, is a visual representation of an organization's management structure and hierarchy of positions**. It helps to understand the distribution of authority, responsibilities, and roles of each department and individual within the organization clearly. Essentially, the Organization Chart acts as a compass that guides employees to better understand their roles, know who they report to, and collaborate effectively with others.

## Types Organization Chart & Structure

Typically, organization charts come in various ty

In [13]:
# เก็บลิงก์ย่อยทั้งหมดที่ดึงมาได้ (เริ่มต้นเป็น list ว่าง)
meta_links = []

# root ของเว็บไซต์ (ใช้ต่อเติมเป็น absolute URL)
root_path = 'https://www.jenosize.com'

# ดึงลิงก์ทั้งหมดจากหน้า "ideas" โดยใช้ฟังก์ชัน get_uri_from
links = get_url_from("https://www.jenosize.com/en/ideas")

# วนลูปผ่านลิงก์ที่ได้มา
for link in links:
    # ต่อ URL แบบเต็ม (root + path) โดยตัด "./" ที่ขึ้นต้นออก (เลยใช้ link[2:])
    travel_link = f"{root_path}{link[2:]}"
    
    # ดึง sub-links จากแต่ละหน้าหมวดหมู่ย่อย
    # pattern ถูกกำหนดให้เจาะจงหา link ที่อยู่ใน path /en/ideas/{category}/
    sub_links = get_url_from(travel_link, pattern=f"a[href*='/en/ideas/{link.split('/')[-1]}/']")
    
    # เพิ่ม sub-links เข้า meta_links โดยใช้ set() เพื่อลดการซ้ำ
    meta_links.extend(set(sub_links))

# แสดงผลลิงก์ทั้งหมดที่เก็บไว้ (พิมพ์ทีละบรรทัด)
print(*meta_links, sep='\n')


/en/ideas/futurist/experiential-marketing-2030-trends
/en/ideas/futurist/ai-customer-service-platform
/en/ideas/futurist/behavioral-design-for-marketing
/en/ideas/futurist/laude-4-sonnet-thai-prompts
/en/ideas/futurist/cloud-computing-trends
/en/ideas/futurist/ai-search-marketing-strategies
/en/ideas/futurist/deepseek-ai-chatbot
/en/ideas/futurist/hello-jenie-ai
/en/ideas/futurist/japan-ai-earthquake-disaster-response
/en/ideas/futurist/agentic-ai-guide-for-modern-business
/en/ideas/futurist/digital-twin-business-model
/en/ideas/futurist/business-model-canvas-2-0
/en/ideas/utility-for-our-world/what-is-e-waste
/en/ideas/utility-for-our-world/5-green-energies-for-businesses
/en/ideas/utility-for-our-world/carbon-credits-business-opportunities
/en/ideas/utility-for-our-world/fast-fashion-trend
/en/ideas/utility-for-our-world/esg-green-technology
/en/ideas/utility-for-our-world/improving-sustainability-in-business
/en/ideas/utility-for-our-world/global-risks-business-preparedness
/en/idea

## Scraping Content with Selenium and BeautifulSoup

In [14]:
# กำหนด root ของเว็บไซต์
root_path = 'https://www.jenosize.com'

# เก็บผลลัพธ์การ scraping ทั้งหมด
output = []

# วนลูปผ่านทุกลิงก์ที่ได้มาจาก meta_links
for link in meta_links:
    # เปิด browser ด้วย Selenium
    # *** อย่างที่บอกไปก่อนหน้านี้ เปลี่ยน browser ตามที่คุณต้องการได้เลย แต่ผมแนะนำ Chrome ง่ายและสะดวก แต่อาจกิน resource เครื่องได้
    driver = webdriver.Chrome()
    driver.get(root_path + link)

    # รอจนกว่า element <section> จะปรากฏ (สูงสุด 60 วินาที)
    # เพื่อให้แน่ใจว่าเนื้อหาหน้าเว็บโหลดเสร็จจริง ๆ
    wait = WebDriverWait(driver, 60)
    element = wait.until(EC.visibility_of_element_located((By.TAG_NAME, "section")))

    # ดึง HTML ของหน้าที่โหลดแล้วมาแปลงเป็น BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ปิด browser เพื่อลดการใช้ resource
    driver.close()

    # log แสดงว่ากำลัง scrape URL ไหนอยู่
    print(f"scraping {root_path+link} ...")

    # ใช้ฟังก์ชัน get_information() ที่เราสร้างไว้
    # เพื่อดึงข้อมูล topic, tags, content ออกมา
    info = get_information(soup)

    # เพิ่ม field 'website' เพื่อเก็บ URL ต้นทาง
    info['website'] = root_path + link

    # เก็บข้อมูลที่ได้ลงใน list output
    output.append(info)

    # หน่วงเวลา 15 วินาที (ป้องกัน block จาก server หรือ rate limit)
    time.sleep(15)

    break # ใช้สำหรับทดสอบ comment คำสั่งนี้เมื่อกำลังจะเริ่มทำงาน loop จริง


scraping https://www.jenosize.com/en/ideas/futurist/experiential-marketing-2030-trends ...


In [18]:
print(f'len: {len(output)}')
print(f'example: {output[:2]}')

len: 1
example: [{'topic': '9 Experiential Marketing Trends to Watch in 2030', 'tags': ['Technology', 'Marketing', 'Business'], 'content': "# 9 Experiential Marketing Trends 2030 Businesses Must Know\n\nCustomer experience is no longer just about providing good service. It has become the cornerstone of marketing strategy across all sectors—whether in SMEs, retail businesses, or government organizations. But have you ever wondered how experiential marketing trends in 2030 will evolve? With cutting-edge technology as a key driver and consumers expecting more complex, higher standards from brands, the landscape is set for a dramatic shift.\n\n\n\n# **9 Experiential Marketing Trends 2030: What Consumers Will Expect in the Future ?**\n\n## **1. Merging the Physical and Digital Worlds (Merged Reality)**\nMerged Reality is set to eliminate the divide between “offline” and “online” experiences. Using Mixed Reality (MR) technology, consumers will engage with brands through a hybrid of physical 

In [19]:
# กำหนด path ของไฟล์ output ที่จะเก็บข้อมูล JSON
output_path = Path("../datasets/jenosize-article.json")

# เปิดไฟล์ในโหมดเขียน ('w') เพื่อเตรียมเขียนข้อมูลลงไป
fp = output_path.open('w')

# บันทึกข้อมูล output (list ของ dicts) ลงในไฟล์ JSON
# ensure_ascii=False ทำให้เก็บตัวอักษรภาษาไทย/Unicode ได้ตรง ๆ
json.dump(output, fp, ensure_ascii=False)


### Analyzing and Generating Content with LLM
หลังจากที่เราได้ทำการดึงข้อมูลและบันทึกบทความให้อยู่ในรูปแบบ JSON เรียบร้อยแล้ว  
ขั้นตอนต่อไปคือการนำ Large Language Model (LLM) มาช่วยในการวิเคราะห์เนื้อหาที่ได้มา  
พร้อมทั้งสร้างข้อความใหม่ที่ผ่านการประมวลผล และจัดเตรียมผลลัพธ์เพื่อบันทึกออกมาเป็นไฟล์ JSON ชุดใหม่

In [20]:
jenosize_fp = Path('../datasets/jenosize-article.json').open('r')

jenosize_contents = json.load(jenosize_fp)
jenosize_contents[0]

{'topic': '9 Experiential Marketing Trends to Watch in 2030',
 'tags': ['Technology', 'Marketing', 'Business'],
 'content': "# 9 Experiential Marketing Trends 2030 Businesses Must Know\n\nCustomer experience is no longer just about providing good service. It has become the cornerstone of marketing strategy across all sectors—whether in SMEs, retail businesses, or government organizations. But have you ever wondered how experiential marketing trends in 2030 will evolve? With cutting-edge technology as a key driver and consumers expecting more complex, higher standards from brands, the landscape is set for a dramatic shift.\n\n\n\n# **9 Experiential Marketing Trends 2030: What Consumers Will Expect in the Future ?**\n\n## **1. Merging the Physical and Digital Worlds (Merged Reality)**\nMerged Reality is set to eliminate the divide between “offline” and “online” experiences. Using Mixed Reality (MR) technology, consumers will engage with brands through a hybrid of physical and digital exp

In [ ]:
# ตั้งค่า API key สำหรับเชื่อมต่อกับเซิร์ฟเวอร์ LLM (ในที่นี้ใช้ค่า placeholder "EMPTY")
openai_api_key = "EMPTY"

# ตั้งค่า base URL ของ vLLM API server ที่เราจะเรียกใช้
openai_api_base = "http://xx.xxx.xxx.xx/hf-llm/v1"

# สร้าง client สำหรับเชื่อมต่อ LLM API
# client นี้จะใช้ส่ง request เพื่อทำการวิเคราะห์หรือสร้างข้อความจากโมเดล
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


In [ ]:
# กำหนด template สำหรับส่งให้ LLM
# - ระบุว่าผลลัพธ์ต้องเป็น string ล้อมด้วย double quotes
# - วิเคราะห์บทความที่ให้มา
# - ห้ามมี reasoning, explanation, markdown หรือ backticks
# - ต้องคืนค่า Python dict ที่มี keys ตามที่กำหนด: "topic_category", "industry", "target_audience", "seo_keywords"
template = \
"""
* Output must be a string only around with double quote only.
* Analyse the given article.
* No reasoning, no explanation, no markdown, no backticks.
* Return a Python dict with these exact keys only: "topic_category", "industry", "target_audience", "seo_keywords".
assistant: 
{{ 
    'topic_category': ['technology', 'fashion', 'finance', 'health'],
    'target_audience': ['Gen Z consumers', 'business executives', 'AI Engineer'],
    'industry': ['banking', 'retail'],
    'seo_keywords': ['AI trends in education', 'best travel apps 2025'],
}}

<|RAW_TEXT|>
topic: {topic}
tag: {tags}
content: {content}
<|/RAW_TEXT|>

"""

# วน loop ผ่านแต่ละบทความใน jenosize_contents
for idx, content in enumerate(jenosize_contents):
    
    # ส่ง request ไปยัง LLM ผ่าน client.chat.completions.create
    # - model: โมเดลที่จะใช้ (ตัวอย่างเช่น google/gemma-3-12b-it)
    # - messages: conversation format ของ LLM (system + user)
    # - user content: template ที่ format ด้วยข้อมูลของบทความ
    # - max_tokens: จำนวน token สูงสุดที่ตอบกลับ
    # - temperature: ความสุ่มของการตอบ (0.05 ≈ deterministic)
    chat_response = client.chat.completions.create(
        model="google/gemma-3-12b-it",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": template.format(**content)},
        ],
        max_tokens=20000,
        temperature=0.05
    )

    # แสดงผลลัพธ์จาก LLM
    # eval() ใช้แปลง string dict ที่ LLM คืนค่าเป็น Python dict จริง
    print("Chat response:", eval(chat_response.choices[0].message.content), sep='\n')

    # อัปเดตข้อมูลบทความใน jenosize_contents ด้วยผลลัพธ์จาก LLM
    jenosize_contents[idx].update(eval(chat_response.choices[0].message.content))
    
    break  # ใช้สำหรับทดสอบ comment คำสั่งนี้เมื่อกำลังจะเริ่มทำงาน loop จริง

In [ ]:
# กำหนด path ของไฟล์ output ที่จะเก็บข้อมูล JSON ใหม่
# ในที่นี้เป็นไฟล์ที่มีผลลัพธ์จากการวิเคราะห์ด้วย LLM

output_path = Path("../datasets/jenosize-article-mockup.json")

fp = output_path.open('w')

json.dump(jenosize_contents, fp, ensure_ascii=False)
